In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

import importlib
import utils

In [ ]:
importlib.reload(utils)

try:
    shuffled_cnn = utils.load_file_pickle("./shuffled_cnn.pkl")
except:
    print("Can't find shuffled_cnn.pkl")
    shuffled_cnn = []
for _ in range(20):
    net = utils.CNN().cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    trainer = utils.CIFAR_trainer(data_params = dict(
                                    pixel_shuffled = True
                                 ),
                                  train_params = dict(batch_size = 4,
                                      num_epochs = 20),
                model_optim_params = dict(model = net, criterion = criterion, optimizer = optimizer))
    trainer.train()
    shuffled_cnn.append(trainer.test())
    utils.save_file_pickle("shuffled_cnn.pkl", shuffled_cnn)

[1,  2000] loss: 2.199
[1,  4000] loss: 2.053
[1,  6000] loss: 1.945
[1,  8000] loss: 1.912
[1, 10000] loss: 1.853
[1, 12000] loss: 1.813
[2,  2000] loss: 1.745
[2,  4000] loss: 1.718
[2,  6000] loss: 1.685
[2,  8000] loss: 1.675
[2, 10000] loss: 1.658
[2, 12000] loss: 1.650
[3,  2000] loss: 1.604
[3,  4000] loss: 1.572
[3,  6000] loss: 1.613
[3,  8000] loss: 1.580
[3, 10000] loss: 1.563
[3, 12000] loss: 1.560
[4,  2000] loss: 1.534
[4,  4000] loss: 1.521
[4,  6000] loss: 1.530
[4,  8000] loss: 1.524
[4, 10000] loss: 1.534
[4, 12000] loss: 1.520
[5,  2000] loss: 1.476
[5,  4000] loss: 1.466
[5,  6000] loss: 1.487
[5,  8000] loss: 1.485
[5, 10000] loss: 1.494
[5, 12000] loss: 1.494
[6,  2000] loss: 1.412
[6,  4000] loss: 1.464
[6,  6000] loss: 1.450
[6,  8000] loss: 1.465
[6, 10000] loss: 1.456
[6, 12000] loss: 1.458
[7,  2000] loss: 1.410
[7,  4000] loss: 1.421
[7,  6000] loss: 1.411
[7,  8000] loss: 1.411
[7, 10000] loss: 1.438
[7, 12000] loss: 1.432
[8,  2000] loss: 1.396
[8,  4000] 

[18, 10000] loss: 1.210
[18, 12000] loss: 1.222
[19,  2000] loss: 1.103
[19,  4000] loss: 1.140
[19,  6000] loss: 1.165
[19,  8000] loss: 1.188
[19, 10000] loss: 1.182
[19, 12000] loss: 1.218
[20,  2000] loss: 1.094
[20,  4000] loss: 1.106
[20,  6000] loss: 1.125
[20,  8000] loss: 1.176
[20, 10000] loss: 1.176
[20, 12000] loss: 1.185
Finished Training
Test loss: 1.748
[1,  2000] loss: 2.206
[1,  4000] loss: 2.048
[1,  6000] loss: 1.983
[1,  8000] loss: 1.925
[1, 10000] loss: 1.885
[1, 12000] loss: 1.832
[2,  2000] loss: 1.784
[2,  4000] loss: 1.762
[2,  6000] loss: 1.714
[2,  8000] loss: 1.705
[2, 10000] loss: 1.702
[2, 12000] loss: 1.655
[3,  2000] loss: 1.613
[3,  4000] loss: 1.628
[3,  6000] loss: 1.608
[3,  8000] loss: 1.598
[3, 10000] loss: 1.599
[3, 12000] loss: 1.595
[4,  2000] loss: 1.556
[4,  4000] loss: 1.543
[4,  6000] loss: 1.547
[4,  8000] loss: 1.536
[4, 10000] loss: 1.538
[4, 12000] loss: 1.529
[5,  2000] loss: 1.485
[5,  4000] loss: 1.484
[5,  6000] loss: 1.487
[5,  800

[16,  2000] loss: 1.115
[16,  4000] loss: 1.148
[16,  6000] loss: 1.157
[16,  8000] loss: 1.195
[16, 10000] loss: 1.193
[16, 12000] loss: 1.229
[17,  2000] loss: 1.095
[17,  4000] loss: 1.129
[17,  6000] loss: 1.145
[17,  8000] loss: 1.180
[17, 10000] loss: 1.196
[17, 12000] loss: 1.194
[18,  2000] loss: 1.069
[18,  4000] loss: 1.117
[18,  6000] loss: 1.120
[18,  8000] loss: 1.160
[18, 10000] loss: 1.168
[18, 12000] loss: 1.203
[19,  2000] loss: 1.075
[19,  4000] loss: 1.091
[19,  6000] loss: 1.109
[19,  8000] loss: 1.144
[19, 10000] loss: 1.162
[19, 12000] loss: 1.164
[20,  2000] loss: 1.035
[20,  4000] loss: 1.077
[20,  6000] loss: 1.104
[20,  8000] loss: 1.124
[20, 10000] loss: 1.138
[20, 12000] loss: 1.166
Finished Training
Test loss: 1.743
[1,  2000] loss: 2.215
[1,  4000] loss: 2.039
[1,  6000] loss: 1.964
[1,  8000] loss: 1.909
[1, 10000] loss: 1.871
[1, 12000] loss: 1.814
[2,  2000] loss: 1.780
[2,  4000] loss: 1.711
[2,  6000] loss: 1.698
[2,  8000] loss: 1.664
[2, 10000] loss

[13,  6000] loss: 1.239
[13,  8000] loss: 1.276
[13, 10000] loss: 1.256
[13, 12000] loss: 1.291
[14,  2000] loss: 1.196
[14,  4000] loss: 1.199
[14,  6000] loss: 1.242
[14,  8000] loss: 1.246
[14, 10000] loss: 1.252
[14, 12000] loss: 1.262
[15,  2000] loss: 1.152
[15,  4000] loss: 1.198
[15,  6000] loss: 1.212
[15,  8000] loss: 1.206
[15, 10000] loss: 1.236
[15, 12000] loss: 1.248
[16,  2000] loss: 1.133
[16,  4000] loss: 1.191
[16,  6000] loss: 1.186
[16,  8000] loss: 1.218
[16, 10000] loss: 1.218
[16, 12000] loss: 1.208
[17,  2000] loss: 1.101
[17,  4000] loss: 1.154
[17,  6000] loss: 1.183
[17,  8000] loss: 1.200
[17, 10000] loss: 1.209
[17, 12000] loss: 1.204
[18,  2000] loss: 1.100
[18,  4000] loss: 1.117
[18,  6000] loss: 1.166
[18,  8000] loss: 1.191
[18, 10000] loss: 1.162
[18, 12000] loss: 1.211
[19,  2000] loss: 1.069
[19,  4000] loss: 1.089
[19,  6000] loss: 1.135
[19,  8000] loss: 1.165
[19, 10000] loss: 1.198
[19, 12000] loss: 1.179
[20,  2000] loss: 1.070
[20,  4000] loss

[10, 10000] loss: 1.351
[10, 12000] loss: 1.348
[11,  2000] loss: 1.281
[11,  4000] loss: 1.268
[11,  6000] loss: 1.306
[11,  8000] loss: 1.312
[11, 10000] loss: 1.325
[11, 12000] loss: 1.305
[12,  2000] loss: 1.248
[12,  4000] loss: 1.262
[12,  6000] loss: 1.284
[12,  8000] loss: 1.289
[12, 10000] loss: 1.275
[12, 12000] loss: 1.303
[13,  2000] loss: 1.207
[13,  4000] loss: 1.254
[13,  6000] loss: 1.231
[13,  8000] loss: 1.267
[13, 10000] loss: 1.290
[13, 12000] loss: 1.264
[14,  2000] loss: 1.180
[14,  4000] loss: 1.197
[14,  6000] loss: 1.227
[14,  8000] loss: 1.236
[14, 10000] loss: 1.248
[14, 12000] loss: 1.287
[15,  2000] loss: 1.154
[15,  4000] loss: 1.190
[15,  6000] loss: 1.208
[15,  8000] loss: 1.208
[15, 10000] loss: 1.240
[15, 12000] loss: 1.242
[16,  2000] loss: 1.140
[16,  4000] loss: 1.181
[16,  6000] loss: 1.179
[16,  8000] loss: 1.178
[16, 10000] loss: 1.219
[16, 12000] loss: 1.232
[17,  2000] loss: 1.111
[17,  4000] loss: 1.150
[17,  6000] loss: 1.174
[17,  8000] loss

[8,  2000] loss: 1.359
[8,  4000] loss: 1.387
[8,  6000] loss: 1.375
[8,  8000] loss: 1.392
[8, 10000] loss: 1.373
[8, 12000] loss: 1.411
[9,  2000] loss: 1.314
[9,  4000] loss: 1.326
[9,  6000] loss: 1.358
[9,  8000] loss: 1.367
[9, 10000] loss: 1.368
[9, 12000] loss: 1.374
[10,  2000] loss: 1.290
[10,  4000] loss: 1.318
[10,  6000] loss: 1.331
[10,  8000] loss: 1.342
[10, 10000] loss: 1.344
[10, 12000] loss: 1.349
[11,  2000] loss: 1.275
[11,  4000] loss: 1.282
[11,  6000] loss: 1.313
[11,  8000] loss: 1.325
[11, 10000] loss: 1.320
[11, 12000] loss: 1.306
[12,  2000] loss: 1.237
[12,  4000] loss: 1.257
[12,  6000] loss: 1.268
[12,  8000] loss: 1.309
[12, 10000] loss: 1.298
[12, 12000] loss: 1.297
[13,  2000] loss: 1.205
[13,  4000] loss: 1.246
[13,  6000] loss: 1.240
[13,  8000] loss: 1.271
[13, 10000] loss: 1.301
[13, 12000] loss: 1.270
[14,  2000] loss: 1.177
[14,  4000] loss: 1.215
[14,  6000] loss: 1.234
[14,  8000] loss: 1.234
[14, 10000] loss: 1.270
[14, 12000] loss: 1.256
[15,

[5,  6000] loss: 1.473
[5,  8000] loss: 1.493
[5, 10000] loss: 1.484
[5, 12000] loss: 1.500
[6,  2000] loss: 1.443
[6,  4000] loss: 1.449
[6,  6000] loss: 1.457
[6,  8000] loss: 1.442
[6, 10000] loss: 1.440
[6, 12000] loss: 1.459
[7,  2000] loss: 1.423
[7,  4000] loss: 1.407
[7,  6000] loss: 1.418


In [ ]:
importlib.reload(utils)

try:
    shuffled_mlp = []#utils.load_file_pickle("shuffled_mlp.pkl")
except:
    shuffled_mlp = []
    
for _ in range(20):
    net = utils.MLP(input_size=(32*32*3), hidden_sizes=[100,100,100],
               activation = "relu").cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    trainer = utils.CIFAR_trainer(data_params = dict(
                                    pixel_shuffled = True
                                 ),
                                  train_params = dict(batch_size = 4,
                                      num_epochs = 20),
                model_optim_params = dict(model = net, criterion = criterion, optimizer = optimizer))
    trainer.train()
    shuffled_mlp.append(trainer.test())
    utils.save_file_pickle("shuffled_mlp.pkl", shuffled_mlp)

[1,  2000] loss: 2.071
[1,  4000] loss: 1.788
[1,  6000] loss: 1.714
[1,  8000] loss: 1.649
[1, 10000] loss: 1.610
[1, 12000] loss: 1.583
[2,  2000] loss: 1.510
[2,  4000] loss: 1.507
[2,  6000] loss: 1.507
[2,  8000] loss: 1.473
[2, 10000] loss: 1.461
[2, 12000] loss: 1.438
[3,  2000] loss: 1.379
[3,  4000] loss: 1.387
[3,  6000] loss: 1.373
[3,  8000] loss: 1.383
[3, 10000] loss: 1.372
[3, 12000] loss: 1.373
[4,  2000] loss: 1.271
[4,  4000] loss: 1.313
[4,  6000] loss: 1.308
[4,  8000] loss: 1.321
[4, 10000] loss: 1.305
[4, 12000] loss: 1.312
[5,  2000] loss: 1.234
[5,  4000] loss: 1.244
[5,  6000] loss: 1.264
[5,  8000] loss: 1.266
[5, 10000] loss: 1.260
[5, 12000] loss: 1.261
[6,  2000] loss: 1.179
[6,  4000] loss: 1.197
[6,  6000] loss: 1.212
[6,  8000] loss: 1.216
[6, 10000] loss: 1.222
[6, 12000] loss: 1.236
[7,  2000] loss: 1.141
[7,  4000] loss: 1.149
[7,  6000] loss: 1.163
[7,  8000] loss: 1.182
[7, 10000] loss: 1.164
[7, 12000] loss: 1.201
[8,  2000] loss: 1.099
[8,  4000] 

[18, 10000] loss: 0.908
[18, 12000] loss: 0.931
[19,  2000] loss: 0.821
[19,  4000] loss: 0.827
[19,  6000] loss: 0.880
[19,  8000] loss: 0.888
[19, 10000] loss: 0.908
[19, 12000] loss: 0.932
[20,  2000] loss: 0.800
[20,  4000] loss: 0.826
[20,  6000] loss: 0.872
[20,  8000] loss: 0.878
[20, 10000] loss: 0.890
[20, 12000] loss: 0.879
Finished Training
Test loss: 1.655
[1,  2000] loss: 2.053
[1,  4000] loss: 1.775
[1,  6000] loss: 1.714
[1,  8000] loss: 1.648
[1, 10000] loss: 1.630
[1, 12000] loss: 1.580
[2,  2000] loss: 1.513
[2,  4000] loss: 1.518
[2,  6000] loss: 1.480
[2,  8000] loss: 1.479
[2, 10000] loss: 1.466
[2, 12000] loss: 1.464
[3,  2000] loss: 1.370
[3,  4000] loss: 1.385
[3,  6000] loss: 1.405
[3,  8000] loss: 1.384
[3, 10000] loss: 1.397
[3, 12000] loss: 1.401
[4,  2000] loss: 1.300
[4,  4000] loss: 1.318
[4,  6000] loss: 1.336
[4,  8000] loss: 1.314
[4, 10000] loss: 1.341
[4, 12000] loss: 1.306
[5,  2000] loss: 1.227
[5,  4000] loss: 1.271
[5,  6000] loss: 1.252
[5,  800

[16,  2000] loss: 0.890
[16,  4000] loss: 0.886
[16,  6000] loss: 0.937
[16,  8000] loss: 0.943
[16, 10000] loss: 0.975
[16, 12000] loss: 0.964
[17,  2000] loss: 0.866
[17,  4000] loss: 0.871
[17,  6000] loss: 0.912
[17,  8000] loss: 0.923
[17, 10000] loss: 0.946
[17, 12000] loss: 0.934
[18,  2000] loss: 0.847
[18,  4000] loss: 0.897
[18,  6000] loss: 0.891
[18,  8000] loss: 0.904
[18, 10000] loss: 0.910
[18, 12000] loss: 0.926
[19,  2000] loss: 0.813
[19,  4000] loss: 0.860
[19,  6000] loss: 0.873
[19,  8000] loss: 0.914
[19, 10000] loss: 0.907
[19, 12000] loss: 0.923
[20,  2000] loss: 0.800
[20,  4000] loss: 0.839
[20,  6000] loss: 0.868
[20,  8000] loss: 0.871
[20, 10000] loss: 0.880
[20, 12000] loss: 0.916
Finished Training
Test loss: 1.561
[1,  2000] loss: 2.073
[1,  4000] loss: 1.800
[1,  6000] loss: 1.699
[1,  8000] loss: 1.668
[1, 10000] loss: 1.623
[1, 12000] loss: 1.570
[2,  2000] loss: 1.513
[2,  4000] loss: 1.501
[2,  6000] loss: 1.508
[2,  8000] loss: 1.467
[2, 10000] loss

[13,  6000] loss: 0.984
[13,  8000] loss: 1.026
[13, 10000] loss: 1.020
[13, 12000] loss: 1.030
[14,  2000] loss: 0.925
[14,  4000] loss: 0.947
[14,  6000] loss: 0.973
[14,  8000] loss: 0.993
[14, 10000] loss: 1.003
[14, 12000] loss: 1.012
[15,  2000] loss: 0.904
[15,  4000] loss: 0.920
[15,  6000] loss: 0.957
[15,  8000] loss: 0.968
[15, 10000] loss: 0.979
[15, 12000] loss: 0.984
[16,  2000] loss: 0.856
[16,  4000] loss: 0.929
[16,  6000] loss: 0.938
[16,  8000] loss: 0.961
[16, 10000] loss: 0.934
[16, 12000] loss: 0.978
[17,  2000] loss: 0.867
[17,  4000] loss: 0.889
[17,  6000] loss: 0.907
[17,  8000] loss: 0.942
[17, 10000] loss: 0.923
[17, 12000] loss: 0.966
[18,  2000] loss: 0.853
[18,  4000] loss: 0.891
[18,  6000] loss: 0.904
[18,  8000] loss: 0.890
[18, 10000] loss: 0.914
[18, 12000] loss: 0.934
[19,  2000] loss: 0.820
[19,  4000] loss: 0.867
[19,  6000] loss: 0.877
[19,  8000] loss: 0.873
[19, 10000] loss: 0.923
[19, 12000] loss: 0.927
[20,  2000] loss: 0.824
[20,  4000] loss

In [ ]:

try:
    unshuffled_cnn = utils.load_file_pickle("unshuffled_cnn.pkl")
except:
    unshuffled_cnn = []
    
for _ in range(20):
    net = utils.CNN().cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    trainer = utils.CIFAR_trainer(data_params = dict(
                                    pixel_shuffled = False
                                 ),
                                  train_params = dict(batch_size = 4,
                                      num_epochs = 20),
                model_optim_params = dict(model = net, criterion = criterion, optimizer = optimizer))
    trainer.train()
    unshuffled_cnn.append(trainer.test())
    utils.save_file_pickle("unshuffled_cnn.pkl", unshuffled_cnn)

In [ ]:
importlib.reload(utils)

try:
    unshuffled_mlp = utils.load_file_pickle("unshuffled_mlp.pkl")
except:
    unshuffled_mlp = []
    
for _ in range(20):
    net = utils.MLP(input_size=(32*32*3), hidden_sizes=[100,100,100],
               activation = "relu").cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    trainer = utils.CIFAR_trainer(data_params = dict(
                                    pixel_shuffled = False
                                 ),
                                  train_params = dict(batch_size = 4,
                                      num_epochs = 20),
                model_optim_params = dict(model = net, criterion = criterion, optimizer = optimizer))
    trainer.train()
    unshuffled_mlp.append(trainer.test())
    utils.save_file_pickle("unshuffled_mlp.pkl", unshuffled_mlp)

[1,  2000] loss: 2.051
[1,  4000] loss: 1.778
[1,  6000] loss: 1.707
[1,  8000] loss: 1.652
[1, 10000] loss: 1.607
[1, 12000] loss: 1.589
[2,  2000] loss: 1.536
[2,  4000] loss: 1.505
[2,  6000] loss: 1.497
[2,  8000] loss: 1.470
[2, 10000] loss: 1.463
[2, 12000] loss: 1.460
[3,  2000] loss: 1.378
[3,  4000] loss: 1.390
[3,  6000] loss: 1.391
[3,  8000] loss: 1.387
[3, 10000] loss: 1.405
[3, 12000] loss: 1.385
[4,  2000] loss: 1.299
[4,  4000] loss: 1.323
[4,  6000] loss: 1.316
[4,  8000] loss: 1.314
[4, 10000] loss: 1.312
[4, 12000] loss: 1.319
[5,  2000] loss: 1.229
[5,  4000] loss: 1.243
[5,  6000] loss: 1.286
[5,  8000] loss: 1.287
[5, 10000] loss: 1.241
[5, 12000] loss: 1.261
[6,  2000] loss: 1.189
[6,  4000] loss: 1.209
[6,  6000] loss: 1.209
[6,  8000] loss: 1.204
[6, 10000] loss: 1.242
[6, 12000] loss: 1.217
[7,  2000] loss: 1.129
[7,  4000] loss: 1.170
[7,  6000] loss: 1.187
[7,  8000] loss: 1.169
[7, 10000] loss: 1.191
[7, 12000] loss: 1.175
[8,  2000] loss: 1.087
[8,  4000] 

In [ ]:
importlib.reload(utils)

try:
    unshuffled_mlp = utils.load_file_pickle("unshuffled_mlp.pkl")
except:
    unshuffled_mlp = []
    
for _ in range(20):
    net = utils.MLP(input_size=(32*32*3), hidden_sizes=[100,100,100],
               activation = "relu").cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    trainer = utils.CIFAR_trainer(data_params = dict(
                                    pixel_shuffled = False
                                 ),
                                  train_params = dict(batch_size = 4,
                                      num_epochs = 20),
                model_optim_params = dict(model = net, criterion = criterion, optimizer = optimizer))
    trainer.train()
    unshuffled_mlp.append(trainer.test())
    utils.save_file_pickle("unshuffled_mlp.pkl", unshuffled_mlp)